In [ ]:
import pandas as pd

# بارگیری دیتاست
data = pd.read_csv('Dataset/train.csv')

# مشاهده چند سطر اول دیتاست
print(data.head())

In [ ]:
from sklearn.preprocessing import LabelEncoder

# تبدیل متغیر کیفی به کمی
label_encoder = {}
le = LabelEncoder()
data['Target'] = le.fit_transform(data['Target'])
label_encoder['Target'] = le

# مشاهده دیتاست پس از تبدیل
print(data.head())


In [ ]:
from sklearn.model_selection import KFold

# تعیین تعداد fold ها
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# @title
# Import necessary libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.drop('Target', axis=1))

# محسابه وزن کلاس ها
class_weights = compute_class_weight('balanced', classes=np.unique(data['Target']), y=data['Target'])
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# ساخت مدل
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(units=128, activation='relu', input_dim=input_dim, kernel_regularizer=l2(0.001)))  # First hidden layer with L2 regularization
    model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
    model.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.001)))  # Second hidden layer with L2 regularization
    model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
    model.add(Dense(units=3, activation='softmax'))  # Output layer for multiclass classification
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# آموزش مدل و پیش بینی
accuracy_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in kf.split(scaled_data):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = data.iloc[train_index]['Target'], data.iloc[test_index]['Target']

    ann = build_ann(input_dim=scaled_data.shape[1])
    ann.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, class_weight=class_weights, validation_data=(X_test, y_test))

    y_pred = np.argmax(ann.predict(X_test), axis=1)

    # محاسبه معیارها
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'ANN - Confusion Matrix:\n{cm}')
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'\n')

    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

# چاپ متوسط معیارها
print(f'\n')
print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1-Score: {np.mean(f1_scores)}')


In [ ]:
# @title
from sklearn.svm import SVC

# مقداردهی اولیه مدل
svm = SVC(kernel='linear', random_state=42)

# آموزش مدل و پیش‌بینی
for train_index, test_index in kf.split(scaled_data):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = data.iloc[train_index]['Target'], data.iloc[test_index]['Target']

    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # محاسبه معیارها
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'SVM - Confusion Matrix:\n{cm}')
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'\n')

    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

# چاپ متوسط معیارها
print(f'\n')
print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1-Score: {np.mean(f1_scores)}')


In [ ]:
# @title
from sklearn.neighbors import KNeighborsClassifier

# مقداردهی اولیه مدل
knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski')

# آموزش مدل و پیش‌بینی
for train_index, test_index in kf.split(scaled_data):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = data.iloc[train_index]['Target'], data.iloc[test_index]['Target']

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    # محاسبه معیارها
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'KNN - Confusion Matrix:\n{cm}')
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'\n')

    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

# چاپ متوسط معیارها
print(f'\n')
print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1-Score: {np.mean(f1_scores)}')



In [ ]:
# @title
from sklearn.tree import DecisionTreeClassifier

# مقداردهی اولیه مدل
dt = DecisionTreeClassifier(random_state=42)

# آموزش مدل و پیش‌بینی
for train_index, test_index in kf.split(scaled_data):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = data.iloc[train_index]['Target'], data.iloc[test_index]['Target']

    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # محاسبه معیارها
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'Decision Tree - Confusion Matrix:\n{cm}')
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'\n')

    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

# چاپ متوسط معیارها
print(f'\n')
print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1-Score: {np.mean(f1_scores)}')



In [ ]:
# @title
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score

# مقیاس‌بندی داده‌ها
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.drop('Target', axis=1))

# تعداد fold ها
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# مقداردهی اولیه مدل
lr = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)

# آموزش مدل و پیش‌بینی
for train_index, test_index in kf.split(scaled_data):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = data.iloc[train_index]['Target'], data.iloc[test_index]['Target']

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # محاسبه معیارها
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'Logistic Regression - Confusion Matrix:\n{cm}')
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'\n')

    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

# چاپ متوسط معیارها
print(f'\n')
print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1-Score: {np.mean(f1_scores)}')
